<a href="https://colab.research.google.com/github/Chia-Yu-Liang-nckumse115/browhattoeat_base44/blob/main/softmax%E5%AF%A6%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

第一部分：介紹 softmax 與「贏者通吃」效果

# Softmax 的「贏者通吃」效果

在分類問題中，softmax 常用來將分數 (logits) 轉換成機率分佈。

softmax 的公式是：

\[
\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_j e^{z_j}}
\]

由於使用了指數運算，最大的分數會被放大得特別明顯，造成 **「贏者通吃」(winner-takes-all)** 的效果。

在這個小實驗中，我們會輸入三個數字，然後觀察經過 softmax 轉換後，哪一個數字的機率變得特別大。


第二部分：實作 softmax 函式

In [ ]:
import numpy as np

def softmax(x):
    """
    計算 softmax
    x: list 或 numpy array
    """
    x = np.array(x)
    # 減去最大值，避免數值爆炸
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)


1.x = np.array(x) 將輸入的 x 轉換成陣列像 [1,2,3]，有助於接下來的運算像是指數、加總   
2.return exp_x / np.sum(exp_x), 對應到 softmax 計算公式

第三部分：互動式輸入

# 互動式觀察 softmax
請輸入三個數字，看看 softmax 的結果。

觀察：
1. 當其中一個數字比其他大很多時，它的 softmax 機率接近 1。
2. 當三個數字差不多時，機率比較平均。


1.ipywidgets: Jupyter/Colab 的互動式工具套件。
可以讓 Notebook 的輸入輸出更直覺，比如拉條滑桿、輸入框、選單，馬上看到結果。

2.interact
ipywidgets 提供的一個函式。
用來快速把 Python 函數包裝成「互動小工具」。

ex:

@interact(x=1)

def f(x):

    print(x)

這樣的代碼會在 Notebook 產生一個輸入框，讓我們修改 x 的值，函數 f 就會自動執行。

In [ ]:
from ipywidgets import interact, FloatText

@interact(a=FloatText(value=1.0), b=FloatText(value=2.0), c=FloatText(value=3.0))
def show_softmax(a, b, c):
    values = [a, b, c]
    probs = softmax(values)
    print(f"輸入數字: {values}")
    print(f"softmax 結果: {probs}")
    print(f"機率總和: {np.sum(probs):.2f}")


interactive(children=(FloatText(value=1.0, description='a'), FloatText(value=2.0, description='b'), FloatText(…

# **數值穩定性**

上面版本的 softmax 太過簡單，於是透過輸入 prompt 想試試看有沒有其他優化 softmax 的方法

prompt: 有什麼優化softmax的辦法

1.減去最大值（max-shift），避免 exp 爆炸

In [ ]:
import numpy as np

def stable_softmax(z):
    z = np.asarray(z)
    z = z - np.max(z, axis=-1, keepdims=True)
    ez = np.exp(z)
    return ez / np.sum(ez, axis=-1, keepdims=True)


2.log-sum-exp 技巧（需要 log 概率時）

In [ ]:
def log_softmax(z):
    z = np.asarray(z)
    z = z - np.max(z, axis=-1, keepdims=True)
    logZ = np.log(np.sum(np.exp(z), axis=-1, keepdims=True))
    return z - logZ  # = log softmax


3.混合精度/FP16 注意事項

先做 max/sum 的歸一化，再 exp。

必要時把歸一化步驟在 FP32 完成，再 cast 回 FP16。

批量較大時小心 sum 的精度（可用 Kahan sum 或框架內建的 reduce）。